In [ ]:
%pip install -U hats lsdb

In [2]:
import math
from pathlib import Path
from shutil import rmtree

import dask
import lsdb
from dask.distributed import Client
from lsdb.core.search.pixel_search import PixelSearch
from tqdm.auto import tqdm

from hats_import_parquet import hats_import_parquet

dask.config.set({
    "distributed.comm.timeouts.connect": "600s",   # Connection timeout
    "distributed.comm.timeouts.tcp": "1200s",       # Communication timeout
    "distributed.nanny.shutdown-timeout": "1200s",  # Increase nanny shutdown timeout
})

/ocean/projects/phy210048p/malanche/zubercal-filtering/cenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# "Global" paths on PSC Bridges2 cluster
GLOBAL_HATS_PATH = Path("/ocean/projects/phy210048p/shared/hats/catalogs/")
LOCAL_HATS_PATH = Path("./hats")

PS1_OTMO_PATH = GLOBAL_HATS_PATH / "ps1/ps1_otmo"
PS1_OTMO_MARGIN_PATH = GLOBAL_HATS_PATH / "ps1/ps1_otmo_10arcs"

ZUBERCAL_PATH = GLOBAL_HATS_PATH / "ztf_dr16/zubercal"

GAIA_CATALOG_TYPE = "vcep"
GAIA_VARS_PATH = LOCAL_HATS_PATH / f"gaia_dr3_{GAIA_CATALOG_TYPE}"
GAIA_VARS_MARGIN_PATH = LOCAL_HATS_PATH / f"gaia_dr3_{GAIA_CATALOG_TYPE}_10arcsec"

PS1_FILTERS = 'grizy'
PS1_MAG_SUFFIXES = ['MeanPSFMag', 'MeanPSFMagErr', 'Flags']

OUTPUT_CATALOG_NAME = f"zubercal_{GAIA_CATALOG_TYPE}"
LSDB_OUTPUT_PATH = Path("./lsdb") / OUTPUT_CATALOG_NAME
LSDB_OUTPUT_PATH.mkdir(exist_ok=True, parents=True)

In [7]:
def matched_catalog(search_filter, output_catalog_name):
    try:
        ps1_otmo = lsdb.read_hats(
            PS1_OTMO_PATH,
            margin_cache=PS1_OTMO_MARGIN_PATH,
            # Few useful columns from PS1 object catalog
            columns=(
                ['objID', 'raMean', 'decMean']
                + [f'{fltr}{suffix}' for fltr in PS1_FILTERS for suffix in PS1_MAG_SUFFIXES]
            ),
            search_filter=search_filter,
        )
    # No coverage
    except ValueError as e:
        print(e)
        return None

    # Zubercal catalog, skip coordinates and few other columns
    # Column description:
    # http://atua.caltech.edu/ZTF/Fields/ReadMe.txt
    try:
        zubercal = lsdb.read_hats(
            ZUBERCAL_PATH,
            columns=['mjd', 'mag', 'magerr', 'objectid', 'band'],
            search_filter=search_filter,
            filters=[
                ("info", "==", 0),  # No errors in calibration
                ("flag", "==", 0),  # Good observational conditions
            ],
        )
    # No coverage
    except ValueError as e:
        print(e)
        return None

    gaia_var = lsdb.read_hats(
        GAIA_VARS_PATH,
        margin_cache=GAIA_VARS_MARGIN_PATH,
        search_filter=search_filter,
    )

    print(gaia_var._ddf.npartitions, ps1_otmo._ddf.npartitions)
    
    # try:
    result = gaia_var.crossmatch(
        ps1_otmo,
        radius_arcsec=1.0,
        suffixes=["", ""],
        output_catalog_name="gaia_vars_x_ps1_otmo",
    ).join_nested(
        zubercal,
        left_on='objID',
        right_on='objectid',
        nested_column_name='lc',
        output_catalog_name=OUTPUT_CATALOG_NAME,
    )
    # except ValueError as e:
    #     print(e)
    #     return None
    
    return result

In [8]:
%%time

order = 1
num_pixels = 12 * 4 ** order
batch_n_digits = len(str(num_pixels))

with Client(n_workers=64, threads_per_worker=1, memory_limit='64GB') as client:
    display(client)

    for pix in tqdm(range(num_pixels)):
        batch_str = f"batch_{pix:0{batch_n_digits}d}"
        output_path = LSDB_OUTPUT_PATH / f"{batch_str}.parquet"
        output_catalog_name = f"{OUTPUT_CATALOG_NAME}_{batch_str}"

        if output_path.exists():
            print(f"{output_path} exists, skipping")
            continue
            # if (properties := output_path / "properties").exists():
            #     print(f"{properties} exists, skipping")
            #     continue
            # print(f"Deleting incomplete catalog {output_path}")
            # rmtree(output_path)

        pixel_search = PixelSearch([(order, pix)])

        print("X-matching...")
        batch = matched_catalog(pixel_search, output_catalog_name)

        # No coverage
        if batch is None:
            print(f"No coverage for tile ({order}, {pix})")
            continue

        print(f"Matched partitions: {batch._ddf.npartitions}")

        print("Computing...")
        result = batch.compute()

        n_rows = result.shape[0]
        print(f"Number of rows: {result.shape[0]}")

        if n_rows == 0:
            print(f"No matched rows for tile ({pix}, {order})")
            continue

        print("Saving...")
        result.drop(["Dir", "Norder", "Npix"], axis=1).to_parquet(output_path, index=True)

/ocean/projects/phy210048p/malanche/zubercal-filtering/cenv/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44501 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:44501/status,
Dashboard: http://127.0.0.1:44501/status,Workers: 64
Total threads: 64,Total memory: 3.73 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40859,Workers: 64
Dashboard: http://127.0.0.1:44501/status,Total threads: 64
Started: Just now,Total memory: 3.73 TiB
Comm: tcp://127.0.0.1:44597,Total threads: 1
Dashboard: http://127.0.0.1:44355/status,Memory: 59.60 GiB
Nanny: tcp://127.0.0.1:37441,


  0%|          | 0/48 [00:00<?, ?it/s]

lsdb/zubercal_vcep/batch_00.parquet exists, skipping
lsdb/zubercal_vcep/batch_01.parquet exists, skipping
lsdb/zubercal_vcep/batch_02.parquet exists, skipping
lsdb/zubercal_vcep/batch_03.parquet exists, skipping
lsdb/zubercal_vcep/batch_04.parquet exists, skipping
lsdb/zubercal_vcep/batch_05.parquet exists, skipping
lsdb/zubercal_vcep/batch_06.parquet exists, skipping
lsdb/zubercal_vcep/batch_07.parquet exists, skipping
lsdb/zubercal_vcep/batch_08.parquet exists, skipping
lsdb/zubercal_vcep/batch_09.parquet exists, skipping
lsdb/zubercal_vcep/batch_10.parquet exists, skipping
lsdb/zubercal_vcep/batch_11.parquet exists, skipping
lsdb/zubercal_vcep/batch_12.parquet exists, skipping
lsdb/zubercal_vcep/batch_13.parquet exists, skipping
lsdb/zubercal_vcep/batch_14.parquet exists, skipping
lsdb/zubercal_vcep/batch_15.parquet exists, skipping
lsdb/zubercal_vcep/batch_16.parquet exists, skipping
lsdb/zubercal_vcep/batch_17.parquet exists, skipping
lsdb/zubercal_vcep/batch_18.parquet exists, sk

 69%|██████▉   | 33/48 [00:00<00:00, 92.73it/s]

The selected sky region has no coverage
No coverage for tile (32, 1)
X-matching...


 69%|██████▉   | 33/48 [00:02<00:00, 15.64it/s]

1 114


ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set

In [ ]:
%%time

hats_import_parquet(LSDB_OUTPUT_PATH, LOCAL_HATS_PATH, OUTPUT_CATALOG_NAME,
                    dask_kwargs={'n_workers': 16, 'memory_limit': '64GB'})

In [2]:
import lsdb
from dask.distributed import Client
from lsdb.core.search.pixel_search import PixelSearch

search_filter = PixelSearch([(1, 33)])
ps1_otmo = lsdb.read_hats(
    "/ocean/projects/phy210048p/shared/hats/catalogs/ps1/ps1_otmo",
    margin_cache="/ocean/projects/phy210048p/shared/hats/catalogs/ps1/ps1_otmo_10arcs",
    columns=(
        ['objID', 'raMean', 'decMean']
    ),
    search_filter=search_filter,
)

gaia_var = lsdb.read_hats(
    "/ocean/projects/phy210048p/malanche/zubercal-filtering/hats/gaia_dr3_vcep",
    search_filter=search_filter,
)

result = gaia_var.crossmatch(
    ps1_otmo,
    radius_arcsec=1.0,
    suffixes=["", ""],
)

ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set